In [1]:
from xml.dom.minidom import parse, parseString
import pandas as pd
import geopy.distance

In [6]:
# the time to start the ride
begin_date = pd.to_datetime("2020-03-24 15:00:00")
# one second
delta = pd.Timedelta(10**9)
# the speed at which the ride is done
speed = 51/3.6

FORMAT = "%Y-%m-%dT%H:%M:%SZ"   # the format for the date
FILE_NAME = "proloog-Venetie"

# extract the xml data from the gpx and all trackpoints
xml = parse(FILE_NAME+".gpx")
trackpoints = xml.getElementsByTagName("trkpt")

# add the start time to the first point
node = trackpoints[0]              # get the first trackpoint
newEle = xml.createElement("time") # create a new element to hold the time
newText= xml.createTextNode(begin_date.strftime(FORMAT))      # create node with the date
newEle.appendChild(newText);       # add this node to the new element
node.appendChild(newEle)           # add the time element to the trackpoint

time = begin_date

for i in range(1, len(trackpoints)):
    node = trackpoints[i]              # get the first trackpoint
    newEle = xml.createElement("time") # create a new element to hold the time
    
    # calculate the distance between this point and the previous
    point0 = trackpoints[i-1]
    point1 = trackpoints[i]
    # extract coordinates
    coord0 = (float(point0.getAttributeNode("lat").nodeValue), float(point0.getAttributeNode("lon").nodeValue))
    coord1 = (float(point1.getAttributeNode("lat").nodeValue), float(point1.getAttributeNode("lon").nodeValue))
    # get the distance in meter
    dist = geopy.distance.geodesic(coord0, coord1).m
    
    # calculate the time between this point and the previous and the time ath this point
    seconds = dist/speed
    time += seconds*delta
    
    newText= xml.createTextNode(time.strftime(FORMAT))      # create node with the date
    newEle.appendChild(newText);       # add this node to the new element
    node.appendChild(newEle)           # add the time element to the trackpoint
    
    
with open(FILE_NAME+"-with-time.gpx", "w") as file:
    print(xml.toprettyxml(), file=file)